In [5]:
%reload_ext autoreload
%autoreload 2

from model.AlexNet import CompositeAlexNet
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

import torch.cuda as cuda

from data_process.DatasetHelper import ImageDataset
import torch.nn as nn

# Functional module contains helper functions
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.transforms as T

from torch.utils.data import random_split, ConcatDataset

### Load the composite AlexNet model

In [6]:
net = CompositeAlexNet(loadkws=dict(map_location=torch.device('cpu')))

### Load the data

In [7]:
tranform_train = T.Compose([
  T.Resize((227,227)),
  # T.ToPILImage(),
  # T.ToTensor(),
  # T.Normalize((133.7900, 122.5296, 116.1627), (78.6071, 76.6452, 77.6168)),
])

data_dir = './data/aug_1'

# unzip the augmented dataset and load it
data = ImageDataset(data_dir, transform=tranform_train)

l0_data = ImageDataset(data_dir, transform=tranform_train)
l0_data.img_labels = data.img_labels[:1000].copy()

l1_data = ImageDataset(data_dir, transform=tranform_train)
l1_data.img_labels = data.img_labels[1000:2000].copy()

l2_data = ImageDataset(data_dir, transform=tranform_train)
l2_data.img_labels = data.img_labels[2000:3000].copy()

l3_data = ImageDataset(data_dir, transform=tranform_train)
l3_data.img_labels = data.img_labels[3000:4000].copy()

l4_data = ImageDataset(data_dir, transform=tranform_train)
l4_data.img_labels = data.img_labels[4000:5000].copy()

d1, d2, d3, d4, d5, test_data = torch.utils.data.random_split(
    l0_data, [180, 180, 180, 180, 180, 100],
    generator=torch.Generator().manual_seed(123)
)

for data in l1_data, l2_data, l3_data, l4_data:
  new_d1, new_d2, new_d3, new_d4, new_d5, new_test_data = torch.utils.data.random_split(
    data, [180, 180, 180, 180, 180, 100],
    generator=torch.Generator().manual_seed(123)
  )
  d1 = ConcatDataset((d1, new_d1))
  d2 = ConcatDataset((d2, new_d2))
  d3 = ConcatDataset((d3, new_d3))
  d4 = ConcatDataset((d4, new_d4))
  d5 = ConcatDataset((d5, new_d5))
  test_data = ConcatDataset((test_data, new_test_data))

batch_size = 40
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

label: 0 with 1000 images, named as "cloth_mask"
label: 1 with 1000 images, named as "no_face_mask"
label: 2 with 1000 images, named as "surgical_mask"
label: 3 with 1000 images, named as "n95_mask"
label: 4 with 1000 images, named as "mask_worn_incorrectly"
label: 0 with 1000 images, named as "cloth_mask"
label: 1 with 1000 images, named as "no_face_mask"
label: 2 with 1000 images, named as "surgical_mask"
label: 3 with 1000 images, named as "n95_mask"
label: 4 with 1000 images, named as "mask_worn_incorrectly"
label: 0 with 1000 images, named as "cloth_mask"
label: 1 with 1000 images, named as "no_face_mask"
label: 2 with 1000 images, named as "surgical_mask"
label: 3 with 1000 images, named as "n95_mask"
label: 4 with 1000 images, named as "mask_worn_incorrectly"
label: 0 with 1000 images, named as "cloth_mask"
label: 1 with 1000 images, named as "no_face_mask"
label: 2 with 1000 images, named as "surgical_mask"
label: 3 with 1000 images, named as "n95_mask"
label: 4 with 1000 image

In [8]:
net.eval()

tensor([-10.2897,  -1.2827,  -4.0100,  -4.7228,  19.8254],
       grad_fn=<SelectBackward0>)
tensor(4)


In [10]:
import evaluation

# example of how to translate my nets output(5 channels) into one label
def output_to_label(output):
  rs = 0
  for i in range(5):
    if output[i] > output[rs]:
      rs = i
  return rs

evaluation.evaluate(
  test_loader, 
  predict_fn=lambda x: net(x.float()),    # example for how my net predicts the output
  output_label_match_fn=output_to_label
)

(0.994,
 0.9940000000000001,
 0.994098233352747,
 0.9939995499587461,
 array([[100,   1,   0,   1,   0],
        [  0,  99,   0,   0,   0],
        [  0,   0, 100,   1,   0],
        [  0,   0,   0,  98,   0],
        [  0,   0,   0,   0, 100]]))